# Reasoning

Chain of thought etc.

## Setup
#### Follow [README](https://github.com/tirtho/open-ai/blob/main/README.md) and perform setup before running the notebooks

Reference : 
- [Azure Open AI](https://learn.microsoft.com/en-us/azure/cognitive-services/openai/overview)

#### Load the API key and relevant Python libaries.

In [36]:
import openai
import sys

from azure_openai_setup import get_openai_client, get_config_from_os_env, get_chat_completion

THE_MODEL = 'gpt-35-turbo-16k'
endpoint, key, version = get_config_from_os_env()
#print(f"{endpoint}, {key}, {version}")
status, client = get_openai_client(aoai_endpoint = endpoint, 
                                   aoai_api_key = key, 
                                   aoai_version = version
                                  )
print(f"Connecting to Open AI returned status as {status}")


Got OPENAI API Key from environment variable
Connecting to Open AI returned status as True


## Math Word Problem

Took the probability example from [here](https://math.stackexchange.com/questions/334516/balls-of-3-colours-in-a-bag-probability)

First try, we are just passing the formula in the abstract example. 
So, we are not providing the detailed Chain of Thought steps.
Let's see if the LLM model was able to answer correctly without detailed Chain of Thought!

The correct answer is 3/44. 

In [15]:
prompt = f"""
There are 12 tennis balls in a bag. 3 of them are red, 4 of them \
are green, and 5 of them are blue. John randomly picks 3 balls \
at the same time. \
What is the probability that all three balls are of the same color? \

Answer:
"""

my_prompt = [
              {
                "role": "user", 
                "content": f"{prompt}"
                }
              ]      
tokens_used, finish_reason, completion = get_chat_completion(
                                                the_client=client, 
                                                the_model=THE_MODEL,
                                                the_messages=my_prompt)
#print(f"Completion: {completion}\nTokens used: {tokens_used}\nFinish Reason: {finish_reason}")
print(f'{completion}')

To find the probability that all three balls are of the same color, we need to find the probability of picking 3 red balls, 3 green balls, or 3 blue balls.

The probability of picking 3 red balls is (3/12) * (2/11) * (1/10) = 1/220.

The probability of picking 3 green balls is (4/12) * (3/11) * (2/10) = 1/165.

The probability of picking 3 blue balls is (5/12) * (4/11) * (3/10) = 1/88.

Therefore, the probability that all three balls are of the same color is (1/220) + (1/165) + (1/88) = 11/660.


#### Now I add the Chain of Thought Reasoning
I provide detailed steps to compute the probability.
Let's watch if the LLM can use the Chain of Thought Reasoning to compute the correct answer!

In [16]:
prompt = f"""
The toy store has N kites. \
Of these, p are white, q are beige, \
and r are orange. Troy bought x kites randomly from the store at the same time. \
What is the probability Troy bought all kites of the same color? \

Answer: 
Y! = Y * (Y -1) * (Y - 2)... 3*2*1
C(U,v) = U!/(v!(U-v)!)
Probability of picking x kites from the p white kites = C(p,x)
Probability of picking x kites from the q beige kites = C(q,x)
Probability of picking x kites from the r orange kites = C(r,x)
Probability of picking x kites from the N kites = C(N,x)

So, the answer is (C(p,x) + C(q,x) + C(r,x))/C(N,x)

There are 12 tennis balls in a bag. 3 of them are red, 4 of them \
are green, and 5 of them are blue. John randomly picks 3 balls \
at the same time. \
What is the probability that all three balls are of the same color? \

Answer:
"""

my_prompt = [
              {
                "role": "user", 
                "content": f"{prompt}"
                }
              ]      
tokens_used, finish_reason, completion = get_chat_completion(
                                                the_client=client, 
                                                the_model=THE_MODEL,
                                                the_messages=my_prompt)
#print(f"Completion: {completion}\nTokens used: {tokens_used}\nFinish Reason: {finish_reason}")
print(f'{completion}')

The total number of ways to pick 3 balls from 12 is C(12,3) = 12!/(3!(12-3)!) = 220.

The number of ways to pick 3 red balls is C(3,3) = 1.
The number of ways to pick 3 green balls is C(4,3) = 4.
The number of ways to pick 3 blue balls is C(5,3) = 10.

So, the probability that all three balls are of the same color is (1+4+10)/220 = 15/220 = 3/44.


## Understanding Data Problem
First one below is easy! :-)

In [17]:
prompt = f"""
I bought a shirt from the local retail store on 12/03/2022. \
I bought the wrong size and want to return it to the store for a refund. \
The store has a 10 day return policy. \
I went to return it on 12/15/2022. \
Why did the store refuse to accept the return?

Answer:
"""

my_prompt = [
              {
                "role": "user", 
                "content": f"{prompt}"
                }
              ]      
tokens_used, finish_reason, completion = get_chat_completion(
                                                the_client=client, 
                                                the_model=THE_MODEL,
                                                the_messages=my_prompt)
#print(f"Completion: {completion}\nTokens used: {tokens_used}\nFinish Reason: {finish_reason}")
print(f'{completion}')

The store refused to accept the return because you exceeded the 10-day return policy. Since you purchased the shirt on 12/03/2022 and attempted to return it on 12/15/2022, it was beyond the allowed time frame for returns.


#### This one may not return the right answer.
Right answer is 05/22/2023

In [24]:
prompt = f"""
The release date of the product was 05/01/2023, \
but it was delayed by 10 days to today. \
What is 10 days from tomorrow in the MM/DD/YYYY format?

Answer:
"""

my_prompt = [
              {
                "role": "user", 
                "content": f"{prompt}"
                }
              ]      
tokens_used, finish_reason, completion = get_chat_completion(
                                                the_client=client, 
                                                the_model=THE_MODEL,
                                                the_messages=my_prompt)
#print(f"Completion: {completion}\nTokens used: {tokens_used}\nFinish Reason: {finish_reason}")
print(f'{completion}')

10 days from tomorrow would be 05/12/2023.


#### Let's add some steps and hope it understands the chain of reasoning
Right answer is 05/22/2023

In [37]:
prompt = f"""
Date of my presentation was on 01/01/2022. \
It got deplayed by 5 days. \
Now that is today! \
So, what date is 7 days from tomorrow in the MM/DD/YYYY format?

Answer: 
Original date = 01/01/2022
Today = Original date + 5 days = 01/01/2022 + 5 days = 01/06/2022
Tomorrow = Today + 1 day = 01/06/2022 + 1 day = 01/07/2022
Answer = 7 days from Tomorrow = 01/07/2022 + 7 days = 01/14/2022

The release date of the product was 05/01/2023, \
but it was delayed by 10 days to today. \
What is 10 days from tomorrow in the MM/DD/YYYY format?

Answer:
"""

my_prompt = [
              {
                "role": "user", 
                "content": f"{prompt}"
                }
              ]      
tokens_used, finish_reason, completion = get_chat_completion(
                                                the_client=client, 
                                                the_model=THE_MODEL,
                                                the_messages=my_prompt)
#print(f"Completion: {completion}\nTokens used: {tokens_used}\nFinish Reason: {finish_reason}")
print(f'{completion}')

Original date = 05/01/2023
Today = Original date + 10 days = 05/01/2023 + 10 days = 05/11/2023
Tomorrow = Today + 1 day = 05/11/2023 + 1 day = 05/12/2023
Answer = 10 days from Tomorrow = 05/12/2023 + 10 days = 05/22/2023
